In [1]:
!git clone --branch dev https://github.com/rolysr/medical-knowledge-discoverer


Cloning into 'medical-knowledge-discoverer'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 550 (delta 96), reused 157 (delta 60), pack-reused 339
Receiving objects: 100% (550/550), 2.37 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (243/243), done.


In [2]:
%cd medical-knowledge-discoverer/
!ls

/content/medical-knowledge-discoverer
datasets  main.py  ontology   requirements.txt
LICENSE   models   README.md  utils


In [3]:
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of es-core-news-sm to determine which version is compatible with other requirements. This coul

In [4]:
%%capture
pip install transformers

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

from pathlib import Path
from utils.anntools import Collection
import csv

import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

## Leer Data y crear Dataframe

In [7]:
train_collection = Collection()
for fname in Path("datasets/train").rglob("*.txt"):
    train_collection.load(fname)

test_collection = Collection()
for fname in Path("datasets/test/scenario2-taskA").rglob("*.txt"):
    test_collection.load(fname)

def build_dataframe(collection):
    lb = []
    txt = []
    for sentence in collection.sentences:
        new_sentence = sentence.text.replace('(','').replace(')','')
        txt.append(new_sentence)
        labels = ['O' for i in range(len(new_sentence.split(' ')))]
        keyphrases = sentence.keyphrases
        sentence_list = new_sentence.split(' ')
        count = 0
        k = 0
        for keyphrase in sentence.keyphrases:
            index = new_sentence.find(keyphrase.text)   
            while count + k < index:
                count += len(sentence_list[k])
                k+=1
            keyphrases_list = keyphrase.text.split(' ')
            if k < len(sentence_list) and sentence_list[k] == keyphrases_list[0]:
                labels[k]= 'B-' + keyphrase.label 
                count += len(keyphrases_list[0])
                i = 1
                while i < len(keyphrases_list):
                    labels[k + i] = 'I-' + keyphrase.label 
                    count += len(keyphrases_list[i])
                    i += 1
                k +=i
        lb.append(labels.copy())

    labels = []

    for label_list in lb:
      current_label = ""
      for label in label_list:
        current_label += label + ' '
      labels.append(current_label)

    data = {
        "text":txt,
        "labels":labels
    }


    df = pd.DataFrame(data)
    
    return df

df = build_dataframe(train_collection)
#df_test = build_dataframe(test_collection) # esto hay que arreglarlo para que tenga los labels
#df_test.head()

## Inicializar Tokenizer


In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

## Crear clase Dataset

In [9]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        print(f"lb {lb}")
        txt = df['text'].values.tolist()
        print(f"txt {txt}")
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

        print(f" texts {self.texts[0]}")
        print(f"labels {self.labels[0]}")

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels



## Separar data y definir etiquetas únicas 

In [15]:
unique_labels = {'B-Action', 'B-Concept', 'B-Predicate', 'B-Reference', 'I-Action', 'I-Concept', 'I-Predicate',  'I-Reference', 'O'}

labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.7 * len(df))])



## Construir modelo

In [16]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

## Entrenando Modelo

In [17]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 4
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and a

lb [['O', 'O', 'O', 'O', 'O', 'O', 'B-Action', 'O', 'O', 'B-Concept', 'O', 'O', 'B-Action', 'O', 'B-Action', 'O'], ['O', 'B-Concept', 'O', 'B-Concept', 'O', 'O', 'O', 'B-Action', 'O', 'O', 'B-Concept', 'O', 'O'], ['O', 'B-Concept', 'O', 'O', 'B-Action', 'O', 'O'], ['O', 'O', 'B-Concept', 'I-Concept', 'I-Concept', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Concept', 'I-Concept', 'O', 'O', 'B-Concept', 'O', 'B-Predicate', 'O', 'B-Concept', 'O', 'B-Predicate', 'O', 'O'], ['O', 'B-Concept', 'B-Concept', 'O', 'O', 'B-Concept', 'O', 'O', 'B-Concept', 'I-Concept', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Concept', 'O', 'B-Action', 'B-Concept', 'O', 'B-Action', 'O', 'O', 'O', 'O'], ['B-Reference', 'B-Concept', 'O', 'B-Action', 'O', 'B-Action', 'O'], ['B-Predicate', 'B-Concept', 'O', 'O'], ['O', 'B-Concept', 'I-Concept', 'I-Concept', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Action', 'O', 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


 texts {'input_ids': tensor([[  101, 11518, 96366, 10107, 10312, 10846, 54284, 10107, 10121, 29276,
         10115,   169, 10198, 24647,   169,   171, 83493, 10129,   193, 82243,
         10341, 10110, 11041, 16626,   119,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,  

100%|██████████| 490/490 [1:41:01<00:00, 12.37s/it]


Epochs: 1 | Loss:  0.947 | Accuracy:  0.670 | Val_Loss:  0.749 | Accuracy:  0.736


100%|██████████| 490/490 [1:39:31<00:00, 12.19s/it]


Epochs: 2 | Loss:  0.718 | Accuracy:  0.748 | Val_Loss:  0.690 | Accuracy:  0.776


 60%|██████    | 294/490 [1:00:08<40:05, 12.27s/it]


KeyboardInterrupt: ignored